In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from sklearn import preprocessing

from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
from keras import optimizers
from sklearn.linear_model import LogisticRegression
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = "./dogBreed/"

train = pd.read_csv(join(data_dir, 'labels.csv'))
# print(train.head())

train_image = []
for i in tqdm(range(train.shape[0])):
    img1 = image.load_img(data_dir + '/train/' + train['id'][i]+'.jpg', target_size=(224,224,3))
    img = image.img_to_array(img1) / 255
    train_image.append(img)
X = np.array(train_image)
print(X.shape)

y = train['breed'].values
le = preprocessing.LabelEncoder()
le.fit(y)
y_trans = le.transform(y)

y_final = to_categorical(y_trans)
print(y_final.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_final, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=96,kernel_size=(11,11),input_shape=(224,224,3),strides=(4,4), padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding='valid'))

model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))

model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))

model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Flatten())

model.add(Dense(4096,input_shape=(224*224*3,),kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(4096, kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(120, activation='softmax', kernel_initializer='random_uniform',bias_initializer='zeros'))

model.summary()

In [ ]:
opt = optimizers.SGD(lr=0.01, momentum=0.9, decay=5e-3)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
# m = X_train.shape[0]
# mini_batch_size = 100
# num_iter = 100
# num_mini_batches = m // mini_batch_size
# print(m, num_mini_batches)
# for i in tqdm(range(num_iter)):
#     for j in (range(num_mini_batches)):
#         mini_Xtrain = X_train[j*mini_batch_size:(j+1)*mini_batch_size,]
#         mini_ytrain = y_train[j*mini_batch_size:(j+1)*mini_batch_size,]
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=True)

In [ ]:

resmodel = Sequential()
weights_path = "../keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
resmodel.add(ResNet50(include_top=False, pooling='avg', weights=weights_path))

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./dogBreed"))

# Any results you write to the current directory are saved as output.

In [ ]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Activation, Flatten, Dropout, BatchNormalization

import h5py

In [ ]:
traindf=pd.read_csv('./dogBreed/labels.csv')
testdf=pd.read_csv('./dogBreed/sample_submission.csv')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(13, 6))
traindf['breed'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
def class_percentages(labels):
    class_map={}
    for i in labels:
        if str(i) not in class_map:
            class_map[str(i)]=1
        else:
            class_map[str(i)]+=1
    #     print(class_map)
    return class_map

p=class_percentages(traindf.breed.values)
# print(p)
# for i in p.items():
#     print(i)

print("Class with maximum images-"+str(max(p, key=p.get))+"  "+str(p[max(p, key=p.get)]))
print("Class with maximum images-"+str(min(p,key=p.get)) +"  "+str(p[min(p, key=p.get)]))

In [ ]:
num_classes = 120
resnet_weights_path = "../Keras-Pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(512))
my_new_model.add(Activation('relu'))
my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.applications.resnet50 import preprocess_input
from keras_preprocessing.image import ImageDataGenerator

image_size = 224
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                             rescale=1./255.,
                             horizontal_flip=True,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             validation_split=0.2)

In [ ]:
train_generator=datagen.flow_from_dataframe(
                        dataframe=traindf,
                        directory="./dogBreed/train/",
                        x_col="id",
                        y_col="breed",
                        has_ext=False,
                        subset="training",
                        batch_size=32,
                        seed=50,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
                        dataframe=traindf,
                        directory="./dogBreed/train/",
                        x_col="id",
                        y_col="breed",
                        has_ext=False,
                        subset="validation",
                        batch_size=1,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

In [ ]:
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
                            dataframe=testdf,
                            directory="./dogBreed/test/",
                            x_col="id",
                            y_col=None,
                            has_ext=False,
                            batch_size=1,
                            seed=42,
                            shuffle=False,
                            class_mode=None,
                            target_size=(image_size, image_size))

In [ ]:
STEP_SIZE_TRAIN=train_generator.n
STEP_SIZE_VALID=valid_generator.n

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

my_new_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=3
)